### Q1

In [4]:
import copy

EMPTY = '.'
WHITE = 'W'
BLACK = 'B'
ROWS, COLS = 8, 8


def create_board():
    board = [[EMPTY for _ in range(COLS)] for _ in range(ROWS)]
    for r in range(3):
        for c in range(COLS):
            if (r + c) % 2 == 1:
                board[r][c] = BLACK
    for r in range(5, 8):
        for c in range(COLS):
            if (r + c) % 2 == 1:
                board[r][c] = WHITE
    return board


def print_board(board):
    print("  " + " ".join(map(str, range(COLS))))
    for i, row in enumerate(board):
        print(i, " ".join(row))


def get_moves(board, player):
    direction = -1 if player == WHITE else 1
    moves = []
    captures = []

    for r in range(ROWS):
        for c in range(COLS):
            if board[r][c] == player:
                for dc in [-1, 1]:
                    nr, nc = r + direction, c + dc
                    if 0 <= nr < ROWS and 0 <= nc < COLS and board[nr][nc] == EMPTY:
                        moves.append(((r, c), (nr, nc)))
                    # Capture
                    nr2, nc2 = r + 2*direction, c + 2*dc
                    if 0 <= nr2 < ROWS and 0 <= nc2 < COLS:
                        if board[nr][nc] == (BLACK if player == WHITE else WHITE) and board[nr2][nc2] == EMPTY:
                            captures.append(((r, c), (nr2, nc2)))

    return captures if captures else moves


def apply_move(board, move):
    (r1, c1), (r2, c2) = move
    piece = board[r1][c1]
    new_board = copy.deepcopy(board)
    new_board[r1][c1] = EMPTY
    new_board[r2][c2] = piece
    if abs(r2 - r1) == 2:  # Capture
        new_board[(r1 + r2)//2][(c1 + c2)//2] = EMPTY
    return new_board


def evaluate(board):
    white_count = sum(row.count(WHITE) for row in board)
    black_count = sum(row.count(BLACK) for row in board)
    return black_count - white_count


def minimax(board, depth, alpha, beta, maximizing):
    current_player = BLACK if maximizing else WHITE
    moves = get_moves(board, current_player)

    if depth == 0 or not moves:
        return evaluate(board), None

    best_move = None
    if maximizing:
        max_eval = float('-inf')
        for move in moves:
            result = apply_move(board, move)
            eval_score, _ = minimax(result, depth - 1, alpha, beta, False)
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in moves:
            result = apply_move(board, move)
            eval_score, _ = minimax(result, depth - 1, alpha, beta, True)
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval, best_move


def is_game_over(board):
    return not get_moves(board, WHITE) or not get_moves(board, BLACK)


def main():
    board = create_board()
    print_board(board)

    while not is_game_over(board):
        # Human Move
        print("\nYour move (W):")
        moves = get_moves(board, WHITE)
        if not moves:
            print("No moves available. You lose!")
            break
        for i, move in enumerate(moves):
            print(f"{i}: {move}")
        choice = int(input("Select move index: "))
        move = moves[choice]
        board = apply_move(board, move)
        print(f"Player moves: {move[0]} → {move[1]}")
        print_board(board)

        if is_game_over(board):
            print("AI has no moves. You win!")
            break

        # AI Move
        print("\nAI's move (B):")
        _, ai_move = minimax(board, 4, float('-inf'), float('inf'), True)
        if ai_move:
            board = apply_move(board, ai_move)
            print(f"AI moves: {ai_move[0]} → {ai_move[1]}")
            print_board(board)
        else:
            print("AI cannot move. You win!")
            break

    white_pieces = sum(row.count(WHITE) for row in board)
    black_pieces = sum(row.count(BLACK) for row in board)
    if white_pieces == 0:
        print("You lost. All your pieces are gone.")
    elif black_pieces == 0:
        print("You won! AI has no pieces left.")
    else:
        print("Game over — draw or no legal moves left.")

if __name__ == "__main__":
    main()


  0 1 2 3 4 5 6 7
0 . B . B . B . B
1 B . B . B . B .
2 . B . B . B . B
3 . . . . . . . .
4 . . . . . . . .
5 W . W . W . W .
6 . W . W . W . W
7 W . W . W . W .

Your move (W):
0: ((5, 0), (4, 1))
1: ((5, 2), (4, 1))
2: ((5, 2), (4, 3))
3: ((5, 4), (4, 3))
4: ((5, 4), (4, 5))
5: ((5, 6), (4, 5))
6: ((5, 6), (4, 7))


Player moves: (5, 2) → (4, 1)
  0 1 2 3 4 5 6 7
0 . B . B . B . B
1 B . B . B . B .
2 . B . B . B . B
3 . . . . . . . .
4 . W . . . . . .
5 W . . . W . W .
6 . W . W . W . W
7 W . W . W . W .

AI's move (B):
AI moves: (2, 1) → (3, 0)
  0 1 2 3 4 5 6 7
0 . B . B . B . B
1 B . B . B . B .
2 . . . B . B . B
3 B . . . . . . .
4 . W . . . . . .
5 W . . . W . W .
6 . W . W . W . W
7 W . W . W . W .

Your move (W):
0: ((4, 1), (3, 2))
1: ((5, 4), (4, 3))
2: ((5, 4), (4, 5))
3: ((5, 6), (4, 5))
4: ((5, 6), (4, 7))
5: ((6, 1), (5, 2))
6: ((6, 3), (5, 2))
Player moves: (5, 4) → (4, 5)
  0 1 2 3 4 5 6 7
0 . B . B . B . B
1 B . B . B . B .
2 . . . B . B . B
3 B . . . . . . .
4 . W . . . W . .
5 W . . . . . W .
6 . W . W . W . W
7 W . W . W . W .

AI's move (B):
AI moves: (3, 0) → (5, 2)
  0 1 2 3 4 5 6 7
0 . B . B . B . B
1 B . B . B . B .
2 . . . B . B . B
3 . . . . . . . .
4 . . . . . W . .
5 W . B . . . W .
6 . W . W . W . W
7 W . W . W . W .

Your move (W):
0: ((6, 1), (4, 3))
1: ((6, 3), (4, 

IndexError: list index out of range

### Q2


In [ ]:
def alpha_beta(cards, l, r, is_max_turn, alpha, beta):
    if l > r:
        return 0

    if is_max_turn:
        max_score = float('-inf')
        pick_left = cards[l] + alpha_beta(cards, l + 1, r, False, alpha, beta)
        pick_right = cards[r] + alpha_beta(cards, l, r - 1, False, alpha, beta)
        max_score = max(pick_left, pick_right)
        alpha = max(alpha, max_score)
        if beta <= alpha:
            return max_score
        return max_score
    else:
        if cards[l] < cards[r]:
            return alpha_beta(cards, l + 1, r, True, alpha, beta)
        else:
            return alpha_beta(cards, l, r - 1, True, alpha, beta)

def simulate_game(cards):
    max_score = 0
    min_score = 0
    l, r = 0, len(cards) - 1
    turn = "Max"

    print(f"Initial Cards: {cards}")

    while l <= r:
        if turn == "Max":
            left_score = cards[l] + alpha_beta(cards, l + 1, r, False, float('-inf'), float('inf'))
            right_score = cards[r] + alpha_beta(cards, l, r - 1, False, float('-inf'), float('inf'))

            if left_score >= right_score:
                picked = cards[l]
                l += 1
            else:
                picked = cards[r]
                r -= 1

            max_score += picked
            print(f"Max picks {picked}, Remaining Cards: {cards[l:r+1]}")
            turn = "Min"
        else:
            if cards[l] < cards[r]:
                picked = cards[l]
                l += 1
            else:
                picked = cards[r]
                r -= 1

            min_score += picked
            print(f"Min picks {picked}, Remaining Cards: {cards[l:r+1]}")
            turn = "Max"

    print(f"\nFinal Scores - Max: {max_score}, Min: {min_score}")
    if max_score > min_score:
        print("Winner: Max")
    elif max_score < min_score:
        print("Winner: Min")
    else:
        print("It's a Draw!")

cards = [4, 10, 6, 2, 9, 5]
simulate_game(cards)


Initial Cards: [4, 10, 6, 2, 9, 5]
Max picks 5, Remaining Cards: [4, 10, 6, 2, 9]
Min picks 4, Remaining Cards: [10, 6, 2, 9]
Max picks 10, Remaining Cards: [6, 2, 9]
Min picks 6, Remaining Cards: [2, 9]
Max picks 9, Remaining Cards: [2]
Min picks 2, Remaining Cards: []

Final Scores - Max: 24, Min: 12
Winner: Max


### Q3

In [ ]:
import random

GRID_SIZE = 10
SHIP_SIZES = [3, 2]

def create_empty_grid():
    return [['~'] * GRID_SIZE for _ in range(GRID_SIZE)]

def print_grid(grid):
    print("  " + " ".join(str(i+1) for i in range(GRID_SIZE)))
    for i, row in enumerate(grid):
        print(chr(65 + i), " ".join(row))

def place_ship(grid, ship_size):
    while True:
        orientation = random.choice(['H', 'V'])
        if orientation == 'H':
            row = random.randint(0, GRID_SIZE - 1)
            col = random.randint(0, GRID_SIZE - ship_size)
            if all(grid[row][col + i] == '~' for i in range(ship_size)):
                for i in range(ship_size):
                    grid[row][col + i] = 'S'
                break
        else:
            row = random.randint(0, GRID_SIZE - ship_size)
            col = random.randint(0, GRID_SIZE - 1)
            if all(grid[row + i][col] == '~' for i in range(ship_size)):
                for i in range(ship_size):
                    grid[row + i][col] = 'S'
                break

def place_all_ships(grid):
    for size in SHIP_SIZES:
        place_ship(grid, size)

def parse_coordinate(coord):
    try:
        row = ord(coord[0].upper()) - 65
        col = int(coord[1:]) - 1
        if 0 <= row < GRID_SIZE and 0 <= col < GRID_SIZE:
            return row, col
    except:
        return None
    return None

def is_game_over(grid):
    return all(cell != 'S' for row in grid for cell in row)

def get_adjacent_cells(row, col):
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    adj = []
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE:
            adj.append((r, c))
    return adj

player_board = create_empty_grid()
ai_board = create_empty_grid()
ai_visible_board = create_empty_grid()

place_all_ships(player_board)
place_all_ships(ai_board)

ai_targets = []
ai_visited = set()

print("Welcome to Battleship!")
print("Your grid is hidden, just enter coordinates to attack (e.g., B4)")

while True:
    print("\nYour Turn:")
    print_grid(ai_visible_board)
    move = input("Enter coordinate to attack (e.g., B4): ").strip().upper()
    coords = parse_coordinate(move)
    if not coords:
        print("Invalid input.")
        continue
    r, c = coords
    if ai_visible_board[r][c] != '~':
        print("You already tried this cell.")
        continue

    if ai_board[r][c] == 'S':
        ai_visible_board[r][c] = 'X'
        ai_board[r][c] = 'X'
        print(f"Player attacks: {move} → Hit!")
    else:
        ai_visible_board[r][c] = 'O'
        print(f"Player attacks: {move} → Miss")

    if is_game_over(ai_board):
        print("Congratulations! You sank all the AI's ships!")
        break

    print("\nAI's Turn:")
    if ai_targets:
        r, c = ai_targets.pop(0)
    else:
        while True:
            r, c = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
            if (r, c) not in ai_visited:
                break
    ai_visited.add((r, c))

    if player_board[r][c] == 'S':
        player_board[r][c] = 'X'
        print(f"AI attacks: {chr(65+r)}{c+1} → Hit!")
        ai_targets.extend([cell for cell in get_adjacent_cells(r, c) if cell not in ai_visited])
    else:
        print(f"AI attacks: {chr(65+r)}{c+1} → Miss")

    if is_game_over(player_board):
        print("AI won! All your ships have been sunk.")
        break


Welcome to Battleship!
Your grid is hidden, just enter coordinates to attack (e.g., B4)

Your Turn:
  1 2 3 4 5 6 7 8 9 10
A ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
B ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
C ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
D ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
E ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
F ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
G ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
H ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
I ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
J ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
Enter coordinate to attack (e.g., B4): B4
Player attacks: B4 → Miss

AI's Turn:
AI attacks: C4 → Miss

Your Turn:
  1 2 3 4 5 6 7 8 9 10
A ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
B ~ ~ ~ O ~ ~ ~ ~ ~ ~
C ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
D ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
E ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
F ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
G ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
H ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
I ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
J ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
Enter coordinate to attack (e.g., B4): B5
Player attacks: B5 → Miss

AI's Turn:
AI attacks: J8 → Miss

Your Turn:
  1 2 3 4 5 6 7 8 9 10
A ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
B ~ ~ ~ O O ~ ~ ~ ~ ~
C ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
D ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
E ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
F ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
G ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
H ~ ~ ~ ~